# Face anoninymizer project

Process : 
- read the image
- detect the faces
- blur the faces
- save the images

In [1]:
import cv2
import os
import mediapipe as mp
import argparse
from pathlib import Path

## Building face anonymizer on an image

In [49]:
# Read the image + some basic operations
image_path_read = os.path.join('.', "katniss.jpg")
img = cv2.imread(image_path_read)
img = cv2.resize(img, (534, 300))
H, W, _ = img.shape
print(H, W)

# Detect faces
mp_face_detection = mp.solutions.face_detection
# Model selection = 0 : faces close to the camera
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BRD to RGB
    out = face_detection.process(img_rgb)
    #print(out.detections)
    if out.detections:  # Making sure we have at least one face in the image
        for detection in out.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box
            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height
            
            # Convert the coordinates of the bounding box into an int, as it is a relative bbox
            x1 = int(x1*W) 
            y1 = int(y1*W) 
            w = int(w*W) 
            h = int(h*W) # Had to add 50 here cuz for some reason, it wasn't taking the whole face
            # 2 next lines needed as idk why, but the whole face isn't taken into consideration
            y1 -= 100
            h -= 50
            #cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (0, 255, 0), 3)
            
            # Blur faces
            img[y1:y1+h, x1:x1+w, :] = cv2.blur(img[y1:y1+h, x1:x1+w, :], (30, 30))

            # Display image
            # cv2.imshow('frame', img)
            # cv2.waitKey(0) 
            # cv2.destroyAllWindows()
            
# Save the image : 
cv2.imwrite(os.path.join('.', "output.jpg"), img)            


300 534


True

## Building face anonymizer for video

In [2]:
# Build face anonymizer for video
def process_image(img, face_detection):
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img_rgb)

    if out.detections:  
        for detection in out.detections:
            location_data = detection.location_data
            bbox = location_data.relative_bounding_box
            x1, y1, w, h = bbox.xmin, bbox.ymin, bbox.width, bbox.height
            
            # Convert the coordinates of the bounding box into an int, as it is a relative bbox
            x1 = int(x1*W) 
            y1 = int(y1*H) 
            w = int(w*W) 
            h = int(h*H) 
            
            # Not needed for all the images/vids
            # y1 -= 100
            # h -= 50
            
            # Blur faces
            img[y1:y1+h, x1:x1+w, :] = cv2.blur(img[y1:y1+h, x1:x1+w, :], (30, 30))
            
    return img

In [3]:
#file_path = os.path.join('.', "katniss.jpg")
video_path = os.path.join('.', "my_vid.mp4")
desired_mode = 'video'

args = argparse.ArgumentParser()
args.add_argument("--mode", default=desired_mode) # Mode = mode selection depending on what the user wants to anonymate the faces on (video, webcam, image)
args.add_argument("--filePath", default=video_path)
args = args.parse_args(['--filePath', video_path]) #'--mode', desired_mode,

mp_face_detection = mp.solutions.face_detection

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    if args.mode in ["image"]:
        # Read image
        img = cv2.imread(args.filePath)
        img = cv2.resize(img, (534, 300))
        img = process_image(img, face_detection)
        cv2.imwrite(os.path.join('.', "output_img.jpg"), img)     
    
    elif args.mode in ["video"]:
        vid = cv2.VideoCapture(args.filePath)
        ret, frame = vid.read()
        output_video = cv2.VideoWriter(os.path.join(".", 'output_vid.mp4'),
                                       cv2.VideoWriter_fourcc(*'MP4V'),
                                       25,
                                       (frame.shape[1], frame.shape[0]))
        
        while ret:
            frame = process_image(frame, face_detection)
            output_video.write(frame)
            ret, frame = vid.read()
        
            
        vid.release()  
        output_video.release()
        
    elif args.mode in ["webcam"]:
        cam = cv2.VideoCapture(0)
        ret, frame = cam.read()
        
        while ret:
            frame = process_image(frame, face_detection)
            cv2.imshow("frame", frame)
            cv2.waitKey(25)
            ret, frame = cam.read()
        cam.release()

## Previous tests

In [ ]:
# For another image
image_path_read = os.path.join('.', "image.jpg")
img = cv2.imread(image_path_read)
img = cv2.resize(img, (414, 736))
img = img[175:730, 0:414] # Cropping

In [22]:
file_path = os.path.join('.', "katniss.jpg")
args = argparse.ArgumentParser()
args.add_argument("--mode", default='image') # Mode = mode selection depending on what the user wants to anonymate the faces on (video, webcam, image)
args.add_argument("--filePath", default=file_path)
args = args.parse_args(['--filePath', file_path])

print(f"Attempting to read image from: {args.filePath}")

img = cv2.imread(args.filePath)

if img is None:
    print(f"Error: cv2.imread() returned None. Could not load image from {args.filePath}")
else:
    print(f"Image loaded successfully. Shape: {img.shape}")
    cv2.imshow('frame', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Attempting to read image from: .\katniss.jpg
Image loaded successfully. Shape: (900, 1600, 3)
